#0. Google Driveをマウント

#1. 必要なライブラリのインストール

##1.1. GluonCVのインストール (pip)

In [1]:
!pip install mxnet gluoncv

##1.2 OpenCVのバージョンを4.1.2.3にする。

python-opencvで"cocoa"が見つからないというエラーを解消するため。

https://qiita.com/jmtsn/items/aa73d837b3a8d4158885

In [2]:
!pip install opencv-python==4.1.2.30
!pip install opencv-contrib-python==4.1.2.30

##1.3 進行状況を表示するtqdmと呼ばれるライブラリをダウンロードする。

In [3]:
!pip install tqdm #進行状況を表示するライブラリ #https://blog2.cct-inc.co.jp/blog/python/python_progressbar/

#2. コードを実行

##2.1. フォルダ名を変数にするため、現在の時刻をを取得する。

In [4]:
import datetime
from pytz import timezone
dates=datetime.datetime.now(timezone('Asia/Tokyo'))
exept_microsec=dates.strftime("%Y-%m-%d-%H-%M")
print('[Today : %s]' % str(exept_microsec))

DATATIME=exept_microsec

[Today : 2020-09-03-20-01]


##2.2. データまでのパスを指定する。

In [5]:
INPUT_PATH="/content/drive/My Drive/datasets/"
INPUT_FILE="videoplayback.mp4"

OUTPUT_PATH="/content/drive/My Drive/datasets/" + "PoseEstim_GluonCV_%s/" % DATATIME
OUTPUT_FILE='PoseEstim_%s_from_%s' % (DATATIME, INPUT_FILE)

#出力するフォルダを作成。
import os
os.mkdir(OUTPUT_PATH)

##2.3. 動画ファイルを取得

In [6]:
import os
import cv2

if os.path.isfile(INPUT_PATH+INPUT_FILE) ==True:
  print("%sにおいて、%sの存在が確認できました。" % (INPUT_PATH,INPUT_FILE))
  print("動画の解析を開始します... ")
  print()
else:
  print("%sにおいて、%sの存在が確認できません。パスを見直してください。" % (INPUT_PATH,INPUT_FILE))

cap = cv2.VideoCapture(INPUT_PATH+INPUT_FILE)
#time.sleep(1)  ### letting the camera autofocus
print(cap)

# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# output file name, encoder, fps, size(fit to image size)
width= cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

axes = None
num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

print("fps     : ", fps)
print("width  : ", width)
print("height : ", height)
print("frames : " , num_frames)

#video = cv2.VideoWriter(OUTPUT_PATH + OUTPUT_FILE ,fourcc, fps, (int(height), int(width)))

/content/drive/My Drive/datasets/において、videoplayback.mp4の存在が確認できました。
動画の解析を開始します... 

<VideoCapture 0x7f570b2ce730>
fps     :  59.94005994005994
width  :  1920.0
height :  1080.0
frames :  1323.0


##2.4. モデルの作成

In [7]:
import mxnet as mx
from gluoncv.model_zoo import get_model

ctx = mx.cpu()
detector_name = "ssd_512_mobilenet1.0_coco"
detector = get_model(detector_name, pretrained=True, ctx=ctx)


detector.reset_class(classes=['person'], reuse_weights={'person':'person'})
detector.hybridize()

estimator = get_model('simple_pose_resnet18_v1b', pretrained='ccd24037', ctx=ctx)
estimator.hybridize()

/usr/local/lib/python3.6/dist-packages/mxnet/gluon/block.py:1512: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


##2.5. 画像1枚ずつ姿勢推定をする。

In [8]:
import gluoncv as gcv
from gluoncv.data.transforms.pose import detector_to_simple_pose, heatmap_to_coord
from gluoncv.utils.viz import cv_plot_image, cv_plot_keypoints

import time
import sys
import numpy as np
from tqdm.autonotebook import tqdm

cap = cv2.VideoCapture(INPUT_PATH+INPUT_FILE)

print("姿勢推定を開始します...")
print()
i=0
for i in tqdm(range(int(num_frames))):
    ret, frame = cap.read()
    frame = mx.nd.array(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).astype('uint8')

    x, frame = gcv.data.transforms.presets.ssd.transform_test(frame, short=512, max_size=350)
    x = x.as_in_context(ctx)
    class_IDs, scores, bounding_boxs = detector(x)

    pose_input, upscale_bbox = detector_to_simple_pose(frame, class_IDs, scores, bounding_boxs,
                                                       output_shape=(128, 96), ctx=ctx)
    if len(upscale_bbox) > 0:
        predicted_heatmap = estimator(pose_input)
        pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox) #pred_coordsが座標, confidenceが信頼度を表す。

        #------------------------------------------------------------------------------------------------------------
        #こっから画像に座標を描画する。
        #------------------------------------------------------------------------------------------------------------
        img = cv_plot_keypoints(frame, pred_coords, confidence, class_IDs, 
                                bounding_boxs, scores, box_thresh=0.5, keypoint_thresh=0.2)
        
        # can't read image, escape
        if img is None:
          print("can't read")
          break

        # add
        img= cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(OUTPUT_PATH + "%s.jpg" % (str(i).zfill(4)), img)
        #------------------------------------------------------------------------------------------------------------


    #------------------------------------------------------------------------------------------------------------
    #こっから座標 (pred_coords) と 信頼度 (confidence) を操作可能な状態へ変換する。
    #------------------------------------------------------------------------------------------------------------
    #mxNetのNDArrayからNumpyへ型を変換する。
    pred_coords_v2 = pred_coords.asnumpy()
    confidence_v2=confidence.asnumpy()

    #2つのshapeを揃える。
    pred_coords_v3=np.squeeze(pred_coords_v2, 0)
    confidence_v3=np.squeeze(confidence_v2, 0)
    #------------------------------------------------------------------------------------------------------------
    
    #------------------------------------------------------------------------------------------------------------
    #こっからテンソル型のデータを作成。(特に意味はない。とりあえず作ってみる。)
    #------------------------------------------------------------------------------------------------------------
    #(x座標, y座標, 信頼度)の順に列方向(axes=1)へ結合
    Output_PoseEstim=np.concatenate([pred_coords_v3, confidence_v3], 1)

    #print(Output_PoseEstim)
    if i==0:
      Output_PoseEstim_Tensor=Output_PoseEstim
    
    else:
      Output_PoseEstim_Tensor=np.dstack([Output_PoseEstim_Tensor, Output_PoseEstim])
    #------------------------------------------------------------------------------------------------------------

    time.sleep(0.01)

    #cv2_imshow(img)

print('姿勢推定が完了しました。')
print()

#video.release()

姿勢推定を開始します...



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  



姿勢推定が完了しました。



In [9]:
print(Output_PoseEstim_Tensor.shape)

(17, 3, 1323)


##2.6. 画像を結合して動画に変換する。

In [10]:
# 画像結合
import glob

print("画像を結合して動画を作成します...")
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
video = cv2.VideoWriter(OUTPUT_PATH + OUTPUT_FILE, fourcc, fps, (int(width), int(height)))

images = glob.glob(OUTPUT_PATH + "*.jpg")

from tqdm.autonotebook import tqdm
for i in tqdm(range(len(images))):
    img = cv2.resize(cv2.imread(images[i]),(int(width),int(height)))
    video.write(img) 

video.release()
print("%sにおいて、%sを作成しました。" % (OUTPUT_PATH, OUTPUT_FILE))

画像を結合して動画を作成します...



/content/drive/My Drive/datasets/PoseEstim_GluonCV_2020-09-03-20-01/において、PoseEstim_2020-09-03-20-01_from_videoplayback.mp4を作成しました。


-補足-

ffmpegを使用する場合は、以下をシェルで打つ。

`ffmpeg -r 60 -i %04d.jpg -vf scale=trunc(iw/2)*2:trunc(ih/2)*2 -r 24 out.mp4'`

https://qiita.com/genchi-jin/items/90078b6ec751fdacbc9e

##2.7. 不要なデータを削除する。

In [11]:
from tqdm.autonotebook import tqdm
import os

print('使用済みデータを削除します...')
for i in tqdm(range(len(images))):
    os.remove(images[i])

print()
print('終了')

使用済みデータを削除します...




終了
